## 测试 线性代数相关的 算子

本 Notebook 主要对比 `jax.lax.linalg.*` 与对应 PyTorch 算子的行为。
每个小节都补充：算子作用、输入约束、输出结构。


In [1]:
import jax
import torch
import numpy as np
import jax.numpy as jnp

### 1. 测试 jax.lax.linalg.cholesky

要求输入是正定矩阵。

- 作用：对对称/厄米正定矩阵做 Cholesky 分解。
- 输入：`A`，形状通常为 `(n, n)`，且满足正定。
- 输出：三角矩阵 `L`（或上三角形式），满足 `A = L @ L.T`（复数时是共轭转置）。


In [2]:
def make_spd_matrix_numpy(n, epsilon=1e-5):
    """
    生成一个 (n, n) 的对称正定矩阵
    """
    # 1. 生成随机矩阵 X
    # 使用 randn (正态分布) 或 rand (均匀分布) 都可以
    X = np.random.randn(n, n)

    # 2. 构造 Gram 矩阵: A = X @ X.T
    # 这一步保证了 A 是对称半正定
    A = np.dot(X, X.T)  # 或者 X @ X.T

    # 3. 添加对角线抖动 (Diagonal Jitter)
    # 这一步保证了 A 是严格正定 (特征值全部 > 0)
    # 没有这一步，np.linalg.cholesky 经常会因为浮点误差报错
    A += np.eye(n) * epsilon

    return A

In [3]:
x = make_spd_matrix_numpy(4)
x

array([[ 1.29233014, -1.07772496, -0.80755837,  1.18327702],
       [-1.07772496,  1.99641649, -0.55041199,  0.53031243],
       [-0.80755837, -0.55041199,  4.6592083 ,  0.125947  ],
       [ 1.18327702,  0.53031243,  0.125947  ,  7.76487897]])

In [4]:
j_x = jnp.array(x, dtype=jnp.float32)
t_x = torch.from_numpy(x).to(torch.float32)

In [5]:
j_out = jax.lax.linalg.cholesky(j_x)
t_out = torch.ops.aten.linalg_cholesky.default(t_x)
print(j_out)
print(np.array(t_out))

[[ 1.136807   0.         0.         0.       ]
 [-0.948028   1.0476924  0.         0.       ]
 [-0.7103742 -1.1681545  1.6703268  0.       ]
 [ 1.0408777  1.4480335  1.5307701  1.4971287]]
[[ 1.136807   0.         0.         0.       ]
 [-0.948028   1.0476924  0.         0.       ]
 [-0.7103742 -1.1681545  1.6703268  0.       ]
 [ 1.0408777  1.4480335  1.5307701  1.4971287]]


/var/folders/4q/td31czvd29qc_qk6bh070n680000gn/T/ipykernel_26983/2238215155.py:4: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  print(np.array(t_out))


### 2. jax.lax.linalg.eig

输出通常包含复数。

- 作用：计算一般方阵的特征值/特征向量。
- 输入：`A`，方阵 `(n, n)`。
- 输出：特征值（通常为复数）；以及特征向量（具体返回结构取决于接口参数）。


In [6]:
jax.lax.linalg.eig(j_x)

[Array([0.08237553+0.j, 7.994179  +0.j, 2.7657702 +0.j, 4.8705125 +0.j],      dtype=complex64),
 Array([[ 0.79086876+0.j,  0.1659281 +0.j, -0.5639241 +0.j,
         -0.17024659+0.j],
        [ 0.55102867+0.j,  0.05837741+0.j,  0.8245291 +0.j,
         -0.11450472+0.j],
        [ 0.2103053 +0.j, -0.01264038+0.j, -0.00389678+0.j,
          0.9775463 +0.j],
        [-0.16329598+0.j,  0.98432726+0.j,  0.04611038+0.j,
          0.04804261+0.j]], dtype=complex64),
 Array([[ 0.79086876+0.j,  0.16592813+0.j, -0.56392413+0.j,
         -0.17024672+0.j],
        [ 0.55102867+0.j,  0.05837752+0.j,  0.8245291 +0.j,
         -0.11450443+0.j],
        [ 0.21030538+0.j, -0.01264031+0.j, -0.00389647+0.j,
          0.9775463 +0.j],
        [-0.16329587+0.j,  0.98432726+0.j,  0.04611047+0.j,
          0.04804266+0.j]], dtype=complex64)]

In [7]:
torch.ops.aten.linalg_eig.default(t_x)

(tensor([0.0824+0.j, 7.9942+0.j, 2.7658+0.j, 4.8705+0.j]),
 tensor([[ 0.7909+0.j,  0.1659+0.j, -0.5639+0.j, -0.1702+0.j],
         [ 0.5510+0.j,  0.0584+0.j,  0.8245+0.j, -0.1145+0.j],
         [ 0.2103+0.j, -0.0126+0.j, -0.0039+0.j,  0.9775+0.j],
         [-0.1633+0.j,  0.9843+0.j,  0.0461+0.j,  0.0480+0.j]]))

### 3. jax.lax.linalg.lu

JAX 返回了三个数组：`[LU, Pivots, Permutation]`。
而 PyTorch 的 `lu_factor` 常见返回两个：`[LU, Pivots]`。

- 作用：LU 分解（带主元）。
- 输入：矩阵 `A`（可方阵或一般矩阵）。
- 输出：`LU`（合并后的 L/U 因子）和主元信息（以及在 JAX 中可见的 permutation 信息）。


In [8]:
jax.lax.linalg.lu(j_x)

[Array([[ 1.2923301 , -1.0777249 , -0.80755836,  1.183277  ],
        [ 0.91561514,  1.5170937 ,  0.86535966,  6.6814523 ],
        [-0.6248855 , -0.80671793,  4.852678  ,  6.255407  ],
        [-0.8339393 ,  0.72352767, -0.38122836, -0.9323835 ]],      dtype=float32),
 Array([0, 3, 2, 3], dtype=int32),
 Array([0, 3, 2, 1], dtype=int32)]

In [9]:
torch.ops.aten.linalg_lu_factor.default(t_x)

(tensor([[ 1.2923, -1.0777, -0.8076,  1.1833],
         [ 0.9156,  1.5171,  0.8654,  6.6815],
         [-0.6249, -0.8067,  4.8527,  6.2554],
         [-0.8339,  0.7235, -0.3812, -0.9324]]),
 tensor([1, 4, 3, 4], dtype=torch.int32))

### 4. jax.lax.linalg.qr

- 作用：QR 分解，把矩阵分解为正交/酉矩阵与上三角矩阵。
- 输入：矩阵 `A`，形状 `(m, n)`。
- 输出：`Q, R`，满足 `A = Q @ R`。


In [10]:
jax.lax.linalg.qr(j_x)

(Array([[-0.5847765 ,  0.20009542,  0.04895616, -0.78460276],
        [ 0.48766816, -0.6854759 , -0.02708013, -0.5399717 ],
        [ 0.36541837,  0.45320067, -0.7865769 , -0.20585252],
        [-0.5354302 , -0.5335672 , -0.61495256,  0.22461943]],      dtype=float32),
 Array([[-2.2099555,  1.118741 ,  1.8389472, -4.544863 ],
        [ 0.       , -2.116547 ,  2.2600608, -4.2127542],
        [ 0.       ,  0.       , -3.766907 , -4.830531 ],
        [ 0.       ,  0.       ,  0.       ,  0.5034599]], dtype=float32))

In [11]:
torch.ops.aten.linalg_qr.default(t_x)

(tensor([[-0.5848,  0.2001,  0.0490, -0.7846],
         [ 0.4877, -0.6855, -0.0271, -0.5400],
         [ 0.3654,  0.4532, -0.7866, -0.2059],
         [-0.5354, -0.5336, -0.6150,  0.2246]]),
 tensor([[-2.2100,  1.1187,  1.8389, -4.5449],
         [ 0.0000, -2.1165,  2.2601, -4.2128],
         [ 0.0000,  0.0000, -3.7669, -4.8305],
         [ 0.0000,  0.0000,  0.0000,  0.5035]]))

### 5. jax.lax.linalg.svd

- 作用：奇异值分解。
- 输入：矩阵 `A`，形状 `(m, n)`。
- 输出：`U, S, Vh`，满足 `A = U @ diag(S) @ Vh`。


In [12]:
jax.lax.linalg.svd(j_x)

(Array([[-0.165928  ,  0.17024669, -0.5639241 , -0.79086876],
        [-0.0583774 ,  0.11450427,  0.82452905, -0.55102867],
        [ 0.01264029, -0.97754633, -0.00389687, -0.2103053 ],
        [-0.9843274 , -0.04804263,  0.04611041,  0.16329594]],      dtype=float32),
 Array([7.994177  , 4.870512  , 2.7657685 , 0.08237558], dtype=float32),
 Array([[-0.16592804, -0.05837739,  0.01264036, -0.98432726],
        [ 0.17024678,  0.11450443, -0.97754604, -0.04804251],
        [-0.5639241 ,  0.82452905, -0.00389676,  0.04611042],
        [-0.7908688 , -0.55102867, -0.21030536,  0.16329601]],      dtype=float32))

In [13]:
torch.ops.aten.linalg_svd.default(t_x)

(tensor([[-0.1659,  0.1702, -0.5639, -0.7909],
         [-0.0584,  0.1145,  0.8245, -0.5510],
         [ 0.0126, -0.9775, -0.0039, -0.2103],
         [-0.9843, -0.0480,  0.0461,  0.1633]]),
 tensor([7.9942, 4.8705, 2.7658, 0.0824]),
 tensor([[-0.1659, -0.0584,  0.0126, -0.9843],
         [ 0.1702,  0.1145, -0.9775, -0.0480],
         [-0.5639,  0.8245, -0.0039,  0.0461],
         [-0.7909, -0.5510, -0.2103,  0.1633]]))

### 6. jax.lax.linalg.triangular_solve

- 作用：解三角线性方程组（如 `A @ X = B` 或 `X @ A = B`）。
- 输入：三角矩阵 `A`、右端项 `B`，以及 `upper/lower`、`left_side`、`unit_diagonal` 等选项。
- 输出：解 `X`，形状与 `B` 对应。

注意：JAX 与 PyTorch 对比时，`lower/upper`、`left_side/left`、`unit_diagonal/unitriangular` 必须一一对齐。


In [14]:
# 构造上三角系统：A @ X = B
j_A = jnp.triu(j_x)
j_B = j_x

j_out = jax.lax.linalg.triangular_solve(
    j_A,
    j_B,
    left_side=True,
    lower=False,
    unit_diagonal=False,
)
j_out

Array([[ 0.22484842, -0.18022177, -0.01881926,  0.        ],
       [-0.6292304 ,  0.94877964, -0.00442946,  0.        ],
       [-0.17744458, -0.11998041,  0.99956155,  0.        ],
       [ 0.15238835,  0.06829629,  0.01622009,  1.        ]],      dtype=float32)

In [15]:
t_A = torch.triu(t_x)
t_B = t_x

t_out = torch.ops.aten.linalg_solve_triangular.default(
    t_A,
    t_B,
    upper=True,
    left=True,
    unitriangular=False,
)
t_out

tensor([[ 0.2248, -0.1802, -0.0188,  0.0000],
        [-0.6292,  0.9488, -0.0044,  0.0000],
        [-0.1774, -0.1200,  0.9996,  0.0000],
        [ 0.1524,  0.0683,  0.0162,  1.0000]])

In [16]:
# 对齐参数后，比较数值差异
np.max(np.abs(np.array(j_out) - np.array(t_out)))

/var/folders/4q/td31czvd29qc_qk6bh070n680000gn/T/ipykernel_26983/1222267338.py:2: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  np.max(np.abs(np.array(j_out) - np.array(t_out)))


np.float32(0.0)

### 7. jax.lax.linalg.eigh

- 作用：针对对称/厄米矩阵的特征分解。
- 输入：对称（实数）或厄米（复数）矩阵 `A`，形状 `(n, n)`。
- 输出：实特征值 `w` 与对应特征向量矩阵 `v`，满足 `A @ v = v @ diag(w)`。


In [17]:
jax.lax.linalg.eigh(j_x)

(Array([[ 0.79086894,  0.56392413, -0.17024685,  0.1659281 ],
        [ 0.5510286 , -0.8245293 , -0.11450394,  0.05837733],
        [ 0.21030523,  0.00389706,  0.97754616, -0.01264017],
        [-0.16329597, -0.04611048,  0.04804241,  0.98432726]],      dtype=float32),
 Array([0.08237574, 2.7657678 , 4.870513  , 7.9941764 ], dtype=float32))

In [18]:
torch.ops.aten.linalg_eigh.default(t_x)

(tensor([0.0824, 2.7658, 4.8705, 7.9942]),
 tensor([[ 0.7909,  0.5639, -0.1702,  0.1659],
         [ 0.5510, -0.8245, -0.1145,  0.0584],
         [ 0.2103,  0.0039,  0.9775, -0.0126],
         [-0.1633, -0.0461,  0.0480,  0.9843]]))

### 8. jax.lax.linalg.householder_product

- 作用：根据 Householder 向量与系数重建正交/酉矩阵（常见于 QR 相关流程）。
- 输入：反射向量矩阵（如 `a`）和标量系数（如 `taus`）。
- 输出：由这些 Householder 反射组成的乘积矩阵 `Q`。


In [19]:
j_taus = jnp.arange(4, dtype=jnp.float32)
jax.lax.linalg.householder_product(j_x, j_taus)

Array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.41682947,  1.3042672 ],
       [ 0.        ,  0.550412  , -0.77057207, -0.21409631],
       [-0.        , -0.5303124 , -0.47294384, -1.2448803 ]],      dtype=float32)

In [20]:
t_taus = torch.arange(4, dtype=torch.float32)
torch.ops.aten.linalg_householder_product.default(t_x, t_taus)

tensor([[ 1.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.4168,  1.3043],
        [ 0.0000,  0.5504, -0.7706, -0.2141],
        [-0.0000, -0.5303, -0.4729, -1.2449]])